In [1]:
import pandas as pd

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [3]:
DATADIR = '../data/'
FILE = 'uniswap.csv'

In [4]:
def read_csv(file):
    fin = open(file, 'r')
    data = fin.read()
    fin.close()
    data = data.split('\n')
    data = [line.split(',') for line  in data]
    return data

In [5]:
def convert_data(data):
    final_data = []
    for line in data:
        new_line = [int(elem) if i > 1 else elem for i, elem in enumerate(line)]
        final_data.append(new_line)
    return final_data
        

In [6]:
csv = read_csv(DATADIR + FILE)

In [7]:
columns_name = csv[0]
columns_value = csv[1]
data = convert_data(csv[2:])

In [8]:
data = data[0:100] # limit to 100 txs

In [9]:
columns_name

['Block #',
 'Transactions',
 'Gas Used',
 'STOP',
 'ADD',
 'MUL',
 'SUB',
 'DIV',
 'SDIV',
 'MOD',
 'SMOD',
 'ADDMOD',
 'MULMOD',
 'EXP',
 'SIGNEXTEND',
 '0x0c',
 '0x0d',
 '0x0e',
 '0x0f',
 'LT',
 'GT',
 'SLT',
 'SGT',
 'EQ',
 'ISZERO',
 'AND',
 'OR',
 'XOR',
 'NOT',
 'BYTE',
 'SHL',
 'SHR',
 'SAR',
 '0x1e',
 '0x1f',
 'KECCAK256',
 '0x21',
 '0x22',
 '0x23',
 '0x24',
 '0x25',
 '0x26',
 '0x27',
 '0x28',
 '0x29',
 '0x2a',
 '0x2b',
 '0x2c',
 '0x2d',
 '0x2e',
 '0x2f',
 'ADDRESS',
 'BALANCE',
 'ORIGIN',
 'CALLER',
 'CALLVALUE',
 'CALLDATALOAD',
 'CALLDATASIZE',
 'CALLDATACOPY',
 'CODESIZE',
 'CODECOPY',
 'GASPRICE',
 'EXTCODESIZE',
 'EXTCODECOPY',
 'RETURNDATASIZE',
 'RETURNDATACOPY',
 'EXTCODEHASH',
 'BLOCKHASH',
 'COINBASE',
 'TIMESTAMP',
 'NUMBER',
 'DIFFICULTY',
 'GASLIMIT',
 'CHAINID',
 'SELFBALANCE',
 '0x48',
 '0x49',
 '0x4a',
 '0x4b',
 '0x4c',
 '0x4d',
 '0x4e',
 '0x4f',
 'POP',
 'MLOAD',
 'MSTORE',
 'MSTORE8',
 'SLOAD',
 'SSTORE',
 'JUMP',
 'JUMPI',
 'PC',
 'MSIZE',
 'GAS',
 'JUMPDES

In [10]:
data

[['0xbbcd65',
  '0x5e6d5bf3c26e57f7660a1d8bcb3105fec8f3d6b6ce90cb8c4a714942b2a5027c',
  106777,
  2,
  187,
  39,
  47,
  22,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  56,
  36,
  2,
  0,
  61,
  117,
  177,
  9,
  0,
  3,
  0,
  13,
  5,
  0,
  0,
  0,
  18,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  0,
  0,
  6,
  9,
  27,
  13,
  3,
  0,
  3,
  0,
  6,
  0,
  8,
  1,
  0,
  0,
  0,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  351,
  114,
  126,
  0,
  21,
  10,
  122,
  218,
  0,
  0,
  7,
  288,
  0,
  0,
  0,
  0,
  414,
  348,
  1,
  85,
  4,
  0,
  0,
  1,
  0,
  0,
  0,
  8,
  1,
  19,
  5,
  0,
  0,
  0,
  0,
  76,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  8,
  7,
  0,
  0,
  30,
  239,
  212,
  150,
  102,
  88,
  41,
  17,
  15,
  18,
  9,
  6,
  3,
  1,
  2,
  0,
  0,
  231,
  139,
  85,
  27,
  18,
  14,
  6,
  5,
  8,
  2,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  3,
  0,
  0,
  0,
  0,
  0,


In [11]:
df = pd.DataFrame(data, columns=columns_name)
df.loc[:, (df != 0).any(axis=0)]

,Block #,Transactions,Gas Used,STOP,ADD,MUL,SUB,DIV,EXP,LT,GT,SLT,EQ,ISZERO,AND,OR,NOT,BYTE,SHL,SHR,KECCAK256,ADDRESS,CALLER,CALLVALUE,CALLDATALOAD,CALLDATASIZE,CALLDATACOPY,CODECOPY,EXTCODESIZE,RETURNDATASIZE,RETURNDATACOPY,TIMESTAMP,POP,MLOAD,MSTORE,SLOAD,SSTORE,JUMP,JUMPI,GAS,JUMPDEST,PUSH1,PUSH2,PUSH3,PUSH4,PUSH5,PUSH6,PUSH8,PUSH9,PUSH10,PUSH11,PUSH12,PUSH13,PUSH14,PUSH15,PUSH18,PUSH20,PUSH21,PUSH28,PUSH29,PUSH31,PUSH32,DUP1,DUP2,DUP3,DUP4,DUP5,DUP6,DUP7,DUP8,DUP9,DUP10,DUP11,DUP12,DUP13,DUP14,DUP15,DUP16,SWAP1,SWAP2,SWAP3,SWAP4,SWAP5,SWAP6,SWAP7,SWAP8,SWAP9,SWAP10,SWAP11,SWAP12,SWAP13,SWAP14,SWAP15,LOG1,LOG2,LOG3,LOG4,CALL,RETURN,DELEGATECALL,STATICCALL,REVERT
0,0xbbcd65,0x5e6d5bf3c26e57f7660a1d8bcb3105fec8f3d6b6ce90...,106777,2,187,39,47,22,1,56,36,2,61,117,177,9,3,0,13,5,18,2,6,9,27,13,3,3,6,8,1,2,351,114,126,21,10,122,218,7,288,414,348,1,85,4,0,1,0,0,0,8,1,19,5,0,76,0,8,7,0,30,239,212,150,102,88,41,17,15,18,9,6,3,1,2,0,0,231,139,85,27,18,14,6,5,8,2,3,0,0,0,0,1,1,3,0,4,6,0,3,0
1,0xbbcd65,0xef6102ed1c3a4841aaae5d6fbb0e5cf139584a0776be...,83381,1,210,38,47,26,5,54,32,0,75,127,175,9,5,0,11,3,18,2,7,7,28,10,3,0,4,11,2,2,352,115,128,20,9,110,224,6,267,424,343,1,94,4,0,1,0,0,0,0,0,19,5,0,80,0,9,8,0,31,254,208,157,95,83,37,22,12,17,9,7,5,1,2,0,0,236,156,82,19,14,13,6,4,8,2,3,0,0,0,0,1,0,3,0,3,6,0,3,0
2,0xbbcd65,0x8a7eec8c1e392d0f2d8178731061024bff3f64f4c0c9...,121450,2,195,39,45,25,2,55,31,0,82,127,172,7,3,0,13,3,17,2,9,9,27,11,3,0,6,8,1,2,345,111,120,19,10,104,229,7,261,415,339,1,106,4,0,1,0,0,0,0,0,19,5,0,83,0,8,9,0,28,250,208,149,89,81,35,17,13,18,9,6,3,1,2,0,0,218,135,77,17,13,12,5,5,8,2,3,0,0,0,0,1,1,3,0,4,6,0,3,0
3,0xbbcd65,0xc5ad659bbb28dc475f7cae798ae752c5cc07f5608aaf...,65866,2,158,28,47,17,20,33,15,0,86,93,110,3,4,1,7,2,16,1,8,7,44,12,6,1,5,5,1,1,280,100,123,20,0,82,187,8,220,419,280,0,97,2,1,1,0,0,0,0,1,3,1,0,45,0,1,7,1,20,203,184,81,86,56,25,26,24,18,8,4,2,0,0,0,0,189,109,47,14,19,17,11,4,5,1,2,0,0,0,0,1,0,0,0,6,6,1,1,1
4,0xbbcd65,0x2c015811f7241d46e583b449850af1d9faed32c036a1...,118690,2,183,39,52,22,1,57,33,0,61,115,154,7,3,0,18,5,17,2,9,9,27,13,3,0,6,8,1,2,334,112,122,18,10,104,211,7,261,412,320,1,84,4,0,1,0,0,0,0,0,19,5,0,65,0,8,7,0,29,228,214,146,88,82,36,16,13,18,9,6,3,1,2,0,0,212,124,81,21,13,12,5,5,8,2,3,0,0,0,0,1,1,3,0,4,6,0,3,0
5,0xbbcd65,0x20a4704294faecc2f3569764797fdc4a511cd8abef0d...,118762,2,195,39,47,22,1,57,32,0,64,118,169,7,3,0,13,5,16,2,6,9,27,13,3,0,6,8,1,2,351,111,118,17,10,104,213,7,262,408,322,1,87,4,0,1,0,0,0,0,0,19,5,0,83,0,8,7,0,28,242,212,147,88,84,36,17,13,18,9,6,3,1,2,0,0,213,135,79,17,13,12,5,5,8,2,3,0,0,0,0,1,1,3,0,4,6,0,3,0
6,0xbbcd65,0x32cd15e283acbb68c1f423b661feb49783c1c8914f23...,119028,2,195,39,47,24,1,57,30,0,76,126,171,7,3,0,13,3,16,2,6,9,27,13,3,0,6,8,1,2,351,111,118,17,10,104,223,7,262,406,332,1,99,4,0,1,0,0,0,0,0,19,5,0,83,0,8,9,0,28,252,212,147,88,84,36,17,13,18,9,6,3,1,2,0,0,215,135,79,17,13,12,5,5,8,2,3,0,0,0,0,1,1,3,0,4,6,0,3,0
7,0xbbcd65,0x911c9ece1b6418b58b9ae99a8ffef09dbf6ca018c01e...,145742,4,231,30,59,44,12,60,53,0,82,159,234,10,5,0,9,6,28,4,11,10,35,17,1,1,7,14,2,2,665,122,189,44,13,195,282,10,436,690,495,1,117,4,0,0,0,0,6,0,0,19,5,0,118,0,9,8,0,46,360,277,190,168,86,56,24,11,16,10,16,6,5,1,0,0,388,230,118,44,19,18,4,4,4,1,8,1,0,0,0,1,1,4,0,6,7,0,5,0
8,0xbbcd65,0xbd7c781dff21d7d5fa20734a3400fd58f05de9d8aad4...,159802,3,331,61,87,52,15,94,56,0,138,217,294,15,7,0,22,5,30,5,9,12,42,17,3,0,8,15,2,3,621,192,212,37,14,199,409,11,493,725,624,2,173,6,0,1,0,0,0,0,0,32,8,0,119,0,11,15,0,51,433,352,281,166,155,70,29,25,34,16,12,9,3,4,0,0,406,259,139,36,28,23,11,10,16,4,6,0,0,0,0,2,0,5,0,5,9,0,6,0
9,0xbbcd65,0x9138120879a122d5ce718194d3be30d0f70a2b37583f...,143881,3,360,60,83,48,8,95,62,0,130,215,310,15,7,0,22,7,35,4,12,12,42,19,3,0,8,15,2,3,648,194,229,37,15,196,404,11,505,752,617,2,167,6,0,1,0,0,0,0,0,32,8,0,145,0,11,13,0,52,450,370,275,179,155,73,30,26,32,14,11,9,4,3,0,0,425,270,141,33,26,23,11,10,16,4,6,0,0,0,0,2,0,5,0,5,9,0,6,0


In [12]:
df[['Block #', 'Transactions', 'Gas Used']]

,Block #,Transactions,Gas Used
0,0xbbcd65,0x5e6d5bf3c26e57f7660a1d8bcb3105fec8f3d6b6ce90...,106777
1,0xbbcd65,0xef6102ed1c3a4841aaae5d6fbb0e5cf139584a0776be...,83381
2,0xbbcd65,0x8a7eec8c1e392d0f2d8178731061024bff3f64f4c0c9...,121450
3,0xbbcd65,0xc5ad659bbb28dc475f7cae798ae752c5cc07f5608aaf...,65866
4,0xbbcd65,0x2c015811f7241d46e583b449850af1d9faed32c036a1...,118690
5,0xbbcd65,0x20a4704294faecc2f3569764797fdc4a511cd8abef0d...,118762
6,0xbbcd65,0x32cd15e283acbb68c1f423b661feb49783c1c8914f23...,119028
7,0xbbcd65,0x911c9ece1b6418b58b9ae99a8ffef09dbf6ca018c01e...,145742
8,0xbbcd65,0xbd7c781dff21d7d5fa20734a3400fd58f05de9d8aad4...,159802
9,0xbbcd65,0x9138120879a122d5ce718194d3be30d0f70a2b37583f...,143881


In [13]:
#pd.options.display.max_rows = None
#df[['Block #'] + columns_name[2:]].head().sum()

In [14]:
#df_by_block = df.groupby('Block #')


In [15]:
#df['sum_column'] = df_by_block[['Gas Used']].transform(sum)
#df.head()

In [16]:
#df = df.sort_values("sum_column", ascending=True)
##df = df.drop("sum_column", axis=1)
#df.head()

In [17]:
#df.head()